In [1]:
import threading

# Helper thread to avoid the Spark StreamingContext from blocking Jupyter
        
class StreamingThread(threading.Thread):
    def __init__(self, ssc):
        super().__init__()
        self.ssc = ssc
    def run(self):
        self.ssc.start()
        self.ssc.awaitTermination()
    def stop(self):
        print('----- Stopping... this may take a few seconds -----')
        self.ssc.stop(stopSparkContext=False, stopGraceFully=True)

In [2]:
#Initializing PySpark
from pyspark import SparkContext, SparkConf
from pyspark.sql.session import SparkSession

# #Spark Config
conf = SparkConf().setAppName("sample_app")
sc = SparkContext(conf=conf)
spark = SparkSession(sc)

22/04/14 01:34:45 WARN Utils: Your hostname, pop-os resolves to a loopback address: 127.0.1.1; using 192.168.1.103 instead (on interface wlp1s0)
22/04/14 01:34:45 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
22/04/14 01:34:46 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
22/04/14 01:34:47 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.
22/04/14 01:34:47 WARN Utils: Service 'SparkUI' could not bind on port 4041. Attempting port 4042.
22/04/14 01:34:47 WARN Utils: Service 'SparkUI' could not bind on port 4042. Attempting port 4043.
22/04/14 01:34:47 WARN Utils: Service 'SparkUI' could not bind on port 4043. Attempting port 4044.


In [3]:
from pyspark.streaming import StreamingContext
from pyspark.sql import Row
from pyspark.sql.functions import udf, struct, array, col, lit
from pyspark.sql.types import StringType

In [4]:
globals()['models_loaded'] = False
globals()['my_model'] = None

# Toy predict function. Normally you'd use your loaded globals()['my_model'] here
def predict(df):
    return 'predicted-name-of-channel'

predict_udf = udf(predict, StringType())

def process(time, rdd):
    if rdd.isEmpty():
        return
    
    print("========= %s =========" % str(time))
    
    # Convert to data frame
    df = spark.read.json(rdd)
    df.show()
    
    # Utilize our predict function
    df_withpreds = df.withColumn("pred", predict_udf(
        struct([df[x] for x in df.columns])
    ))
    df_withpreds.show()
    
    # Normally, you wouldn't use a UDF (User Defined Function) Python function to predict as we did here (you can)
    # but an MLlib model you've built and saved with Spark
    # In this case, you need to prevent loading your model in every call to "process" as follows:
    
    # Load in the model if not yet loaded:
    if not globals()['models_loaded']:
        # load in your models here
        globals()['my_model'] = '***' # Replace '***' with:    [...].load('my_logistic_regression')
        globals()['models_loaded'] = True
        
    # And then predict using the loaded model: 
    # df_result = globals()['my_model'].transform(df)
    # df_result.show()

In [5]:
ssc = StreamingContext(sc, 10)

In [6]:
lines = ssc.socketTextStream("localhost", 8080)
lines.foreachRDD(process)

In [7]:
ssc_t = StreamingThread(ssc)
ssc_t.start()

22/04/14 01:35:05 WARN RandomBlockReplicationPolicy: Expecting 1 replicas with only 0 peer/s.
22/04/14 01:35:05 WARN BlockManager: Block input-0-1649892905400 replicated to only 0 peer(s) instead of 1 peers
22/04/14 01:35:06 WARN RandomBlockReplicationPolicy: Expecting 1 replicas with only 0 peer/s.
22/04/14 01:35:06 WARN BlockManager: Block input-0-1649892906400 replicated to only 0 peer(s) instead of 1 peers
22/04/14 01:35:07 WARN RandomBlockReplicationPolicy: Expecting 1 replicas with only 0 peer/s.
22/04/14 01:35:07 WARN BlockManager: Block input-0-1649892907400 replicated to only 0 peer(s) instead of 1 peers
22/04/14 01:35:08 WARN RandomBlockReplicationPolicy: Expecting 1 replicas with only 0 peer/s.
22/04/14 01:35:08 WARN BlockManager: Block input-0-1649892908400 replicated to only 0 peer(s) instead of 1 peers
22/04/14 01:35:09 WARN RandomBlockReplicationPolicy: Expecting 1 replicas with only 0 peer/s.
22/04/14 01:35:09 WARN BlockManager: Block input-0-1649892909400 replicated to

========= 2022-04-14 01:35:10 =========


22/04/14 01:35:11 WARN RandomBlockReplicationPolicy: Expecting 1 replicas with only 0 peer/s.
22/04/14 01:35:11 WARN BlockManager: Block input-0-1649892911400 replicated to only 0 peer(s) instead of 1 peers
22/04/14 01:35:11 WARN RandomBlockReplicationPolicy: Expecting 1 replicas with only 0 peer/s.
22/04/14 01:35:11 WARN BlockManager: Block input-0-1649892911600 replicated to only 0 peer(s) instead of 1 peers
22/04/14 01:35:12 WARN RandomBlockReplicationPolicy: Expecting 1 replicas with only 0 peer/s.
22/04/14 01:35:12 WARN BlockManager: Block input-0-1649892912400 replicated to only 0 peer(s) instead of 1 peers
22/04/14 01:35:12 WARN RandomBlockReplicationPolicy: Expecting 1 replicas with only 0 peer/s.
22/04/14 01:35:12 WARN BlockManager: Block input-0-1649892912600 replicated to only 0 peer(s) instead of 1 peers
22/04/14 01:35:13 WARN RandomBlockReplicationPolicy: Expecting 1 replicas with only 0 peer/s.
22/04/14 01:35:13 WARN BlockManager: Block input-0-1649892913400 replicated to

+----------+--------------------+----------------+-------------+
|   channel|            datetime|         message|     username|
+----------+--------------------+----------------+-------------+
|#asmongold|2022-04-13T23:35:...|              4*|    thetestty|
|#asmongold|2022-04-13T23:35:...|@DingoPWN zacmon|   frippyfrap|
|#asmongold|2022-04-13T23:35:...|              EZ|     juugels_|
|#asmongold|2022-04-13T23:35:...| they scramblin!|linktothejake|
|#asmongold|2022-04-13T23:35:...|              EZ|        mrdrz|
|#asmongold|2022-04-13T23:35:...|     EZ GOT EMMM|        meer_|
+----------+--------------------+----------------+-------------+



22/04/14 01:35:15 WARN RandomBlockReplicationPolicy: Expecting 1 replicas with only 0 peer/s.
22/04/14 01:35:15 WARN BlockManager: Block input-0-1649892915600 replicated to only 0 peer(s) instead of 1 peers
22/04/14 01:35:16 WARN RandomBlockReplicationPolicy: Expecting 1 replicas with only 0 peer/s.
22/04/14 01:35:16 WARN BlockManager: Block input-0-1649892916600 replicated to only 0 peer(s) instead of 1 peers


...

22/04/14 01:35:17 WARN RandomBlockReplicationPolicy: Expecting 1 replicas with only 0 peer/s.
22/04/14 01:35:17 WARN BlockManager: Block input-0-1649892917600 replicated to only 0 peer(s) instead of 1 peers
22/04/14 01:35:18 WARN RandomBlockReplicationPolicy: Expecting 1 replicas with only 0 peer/s.
22/04/14 01:35:18 WARN BlockManager: Block input-0-1649892918600 replicated to only 0 peer(s) instead of 1 peers
22/04/14 01:35:19 WARN RandomBlockReplicationPolicy: Expecting 1 replicas with only 0 peer/s.
22/04/14 01:35:19 WARN BlockManager: Block input-0-1649892919600 replicated to only 0 peer(s) instead of 1 peers


========= 2022-04-14 01:35:20 =========
+----------+--------------------+--------------------+---------------+
|   channel|            datetime|             message|       username|
+----------+--------------------+--------------------+---------------+
|#asmongold|2022-04-13T23:35:...|                 oof|    mydixierack|
|#asmongold|2022-04-13T23:35:...|wtf\r\n:squi___!s...|    tomtheshiba|
|#asmongold|2022-04-13T23:35:...|                  EZ|   maxfaussett1|
|#asmongold|2022-04-13T23:35:...|@Thexppkiller TAR...| shinygoldstarz|
|#asmongold|2022-04-13T23:35:...|              monkaW|          meer_|
|#asmongold|2022-04-13T23:35:...|             waduhek|        deeehoc|
|#asmongold|2022-04-13T23:35:...|            OMEGALUL| tinyangrygnome|
|#asmongold|2022-04-13T23:35:...|theres also a sin...|        mezzops|
|#asmongold|2022-04-13T23:35:...|                  D:|  linktothejake|
|#asmongold|2022-04-13T23:35:...|            OMEGALUL|       miimitan|
|#asmongold|2022-04-13T23:35:...|    

22/04/14 01:35:20 WARN RandomBlockReplicationPolicy: Expecting 1 replicas with only 0 peer/s.
22/04/14 01:35:20 WARN BlockManager: Block input-0-1649892920600 replicated to only 0 peer(s) instead of 1 peers


+----------+--------------------+--------------------+---------------+--------------------+
|   channel|            datetime|             message|       username|                pred|
+----------+--------------------+--------------------+---------------+--------------------+
|#asmongold|2022-04-13T23:35:...|                 oof|    mydixierack|predicted-name-of...|
|#asmongold|2022-04-13T23:35:...|wtf\r\n:squi___!s...|    tomtheshiba|predicted-name-of...|
|#asmongold|2022-04-13T23:35:...|                  EZ|   maxfaussett1|predicted-name-of...|
|#asmongold|2022-04-13T23:35:...|@Thexppkiller TAR...| shinygoldstarz|predicted-name-of...|
|#asmongold|2022-04-13T23:35:...|              monkaW|          meer_|predicted-name-of...|
|#asmongold|2022-04-13T23:35:...|             waduhek|        deeehoc|predicted-name-of...|
|#asmongold|2022-04-13T23:35:...|            OMEGALUL| tinyangrygnome|predicted-name-of...|
|#asmongold|2022-04-13T23:35:...|theres also a sin...|        mezzops|predicted-

22/04/14 01:35:21 WARN RandomBlockReplicationPolicy: Expecting 1 replicas with only 0 peer/s.
22/04/14 01:35:21 WARN BlockManager: Block input-0-1649892921600 replicated to only 0 peer(s) instead of 1 peers
22/04/14 01:35:22 WARN RandomBlockReplicationPolicy: Expecting 1 replicas with only 0 peer/s.
22/04/14 01:35:22 WARN BlockManager: Block input-0-1649892922600 replicated to only 0 peer(s) instead of 1 peers
22/04/14 01:35:23 WARN RandomBlockReplicationPolicy: Expecting 1 replicas with only 0 peer/s.
22/04/14 01:35:23 WARN BlockManager: Block input-0-1649892923600 replicated to only 0 peer(s) instead of 1 peers


In [8]:
ssc_t.stop()

----- Stopping... this may take a few seconds -----


22/04/14 01:35:24 WARN SocketReceiver: Error receiving data
java.net.SocketException: Socket closed
	at java.base/java.net.SocketInputStream.socketRead0(Native Method)
	at java.base/java.net.SocketInputStream.socketRead(SocketInputStream.java:115)
	at java.base/java.net.SocketInputStream.read(SocketInputStream.java:168)
	at java.base/java.net.SocketInputStream.read(SocketInputStream.java:140)
	at java.base/sun.nio.cs.StreamDecoder.readBytes(StreamDecoder.java:284)
	at java.base/sun.nio.cs.StreamDecoder.implRead(StreamDecoder.java:326)
	at java.base/sun.nio.cs.StreamDecoder.read(StreamDecoder.java:178)
	at java.base/java.io.InputStreamReader.read(InputStreamReader.java:181)
	at java.base/java.io.BufferedReader.fill(BufferedReader.java:161)
	at java.base/java.io.BufferedReader.readLine(BufferedReader.java:326)
	at java.base/java.io.BufferedReader.readLine(BufferedReader.java:392)
	at org.apache.spark.streaming.dstream.SocketReceiver$$anon$2.getNext(SocketInputDStream.scala:121)
	at org.a

========= 2022-04-14 01:35:30 =========
+----------+--------------------+--------------------+-----------------+
|   channel|            datetime|             message|         username|
+----------+--------------------+--------------------+-----------------+
|#asmongold|2022-04-13T23:35:...|             LUL LUL|   shelikesraiden|
|#asmongold|2022-04-13T23:35:...|              SNIPER|           relaxv|
|#asmongold|2022-04-13T23:35:...|       lol motorboat|      weirdo_dude|
|#asmongold|2022-04-13T23:35:...|lame\r\n:freejam0...|            dskay|
|#asmongold|2022-04-13T23:35:...|           THE CHIEF|          fubous7|
|#asmongold|2022-04-13T23:35:...|                 LuL|     refinedalien|
|#asmongold|2022-04-13T23:35:...|           KEKW Clap|         hitpoint|
|#asmongold|2022-04-13T23:35:...|Master chief OMEG...|            meer_|
|#asmongold|2022-04-13T23:35:...|            OMEGALUL|          808mate|
|#asmongold|2022-04-13T23:35:...|MAster Chief\r\n:...|    ripleyhrgiger|
|#asmongold